# Hosting Strands Agents with Amazon Bedrock models in Amazon Bedrock AgentCore Runtime

## Overview

In this tutorial we will learn how to host your existing agent, using Amazon Bedrock AgentCore Runtime. We will provide examples using Amazon Bedrock models and non-Bedrock models such as Azure OpenAI and Gemini.


### Tutorial Details


| Information         | Details                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                                   |
| Agent type          | Single                                                                           |
| Agentic Framework   | Strands Agents                                                                   |
| LLM model           | Anthropic Claude Sonnet 3.7                                                        |
| Tutorial components | Hosting agent on AgentCore Runtime. Using Strands Agent and Amazon Bedrock Model |
| Tutorial vertical   | Cross-vertical                                                                   |
| Example complexity  | Easy                                                                             |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3                                     |

### Tutorial Architecture

In this tutorial we will describe how to deploy an existing agent to AgentCore runtime. 

For demonstration purposes, we will  use a Strands Agent using Amazon Bedrock models

In our example we will use a very simple agent with two tools: `get_weather` and `get_time`. 

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="50%"/>
</div>

### Tutorial Key Features

* Hosting Agents on Amazon Bedrock AgentCore Runtime
* Using Amazon Bedrock models
* Using Strands Agents


## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* Strands Agents

In [2]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## Creating your agents and experimenting locally

Before we deploy our agents to AgentCore Runtime, let's develop and run them locally for experimentation purposes.

For production agentic applications we will need to decouple the agent creation process from the agent invocation one. With AgentCore Runtime, we will decorate the invocation part of our agent with the `@app.entrypoint` decorator and have it as the entry point for our runtime. Let's first look how each agent is developed during the experimentation phase.

The architecture here will look as following:

<div style="text-align:left">
    <img src="images/architecture_local.png" width="50%"/>
</div>

In [3]:
%%writefile strands_claude.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from strands.models import BedrockModel

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"


model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = strands_agent_bedrock(json.loads(args.payload))

Writing strands_claude.py


#### Invoking local agent

In [9]:
!python strands_claude.py '{"prompt": "What is the weather now?"}'

I'd be happy to check the current weather for you. Let me get that information for you right away.
Tool #1: weather
The current weather is sunny! It's a beautiful day outside.

## Preparing your agent for deployment on AgentCore Runtime

Let's now deploy our agents to AgentCore Runtime. To do so we need to:
* Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
* Initialize the App in our code with `app = BedrockAgentCoreApp()`
* Decorate the invocation function with the `@app.entrypoint` decorator
* Let AgentCoreRuntime control the running of the agent with `app.run()`

### Strands Agents with Amazon Bedrock model
Let's start with our Strands Agent using Amazon Bedrock model. All the others will work exactly the same.

In [10]:
%%writefile strands_claude.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel

app = BedrockAgentCoreApp()

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"


model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

@app.entrypoint
def strands_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    print("User input:", user_input)
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

Overwriting strands_claude.py


## Interactive Local Exploration

Now let's explore your agent interactively right here in the notebook! This allows you to test and iterate quickly before any deployment.

In [1]:
# Create and test the agent directly in the notebook
from strands import Agent, tool
from strands_tools import calculator
from strands.models import BedrockModel

# Create the same custom tool
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

# Initialize the agent
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model = BedrockModel(model_id=model_id)
agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

print("✅ Agent initialized successfully! Ready for interactive exploration.")

✅ Agent initialized successfully! Ready for interactive exploration.


### ⚠️ **Important Note About AWS Usage**

**YES** - The code above makes **REAL calls to AWS Bedrock Claude** using your local AWS credentials! 

When you run the cell above:
- ✅ Uses your AWS credentials (from `~/.aws/credentials` or environment variables)
- ✅ Makes actual API calls to `us.anthropic.claude-3-7-sonnet-20250219-v1:0` on AWS Bedrock
- ✅ **Costs real money** - each call is billed to your AWS account
- ✅ No Docker, no deployment needed - just direct API calls

This is **pure local development** but with **real AWS services**.

### 💰 **What Costs Money vs. What's Free**

| Component | Cost | Explanation |
|-----------|------|-------------|
| **Jupyter Notebook** | 🆓 **FREE** | Running locally on your machine |
| **Python execution** | 🆓 **FREE** | All local computation |
| **uv package manager** | 🆓 **FREE** | Just manages Python packages |
| **VS Code** | 🆓 **FREE** | Your editor environment |
| | | |
| **AWS Bedrock Claude API calls** | 💸 **COSTS MONEY** | Each `agent("question")` call |
| **AWS AgentCore Runtime** | 💸 **COSTS MONEY** | When deployed to AWS |

**Bottom line:** Jupyter runs locally and is free. Only the actual calls to AWS Bedrock Claude cost money!

## 🚀 Adding Nova Act to Your Agent

**Yes!** You can add Nova Act (Nova SDK) to your AgentCore Runtime agents. Here's how to integrate it with your Strands agent:

### Option 1: Nova Act as a Tool
Add Nova Act as a custom tool in your Strands agent:

```python
from nova_act import NovaAct
from bedrock_agentcore.tools.browser_client import browser_session

@tool
def browse_with_nova(instruction: str, starting_url: str = "https://google.com"):
    """Use Nova Act to browse the web and complete tasks"""
    with browser_session("us-west-2") as browser_client:
        ws_url, headers = browser_client.generate_ws_headers()
        
        with NovaAct(
            cdp_endpoint_url=ws_url,
            cdp_headers=headers,
            nova_act_api_key="your-api-key",
            starting_page=starting_url,
        ) as nova_act:
            result = nova_act.act(instruction)
            return str(result)
```

### Option 2: Direct Nova Act Integration
Replace or enhance your agent with Nova Act capabilities alongside Claude.

In [ ]:
# Example: Enhanced Strands agent with Nova Act browser tool
%%writefile enhanced_strands_claude_with_nova.py
from strands import Agent, tool
from strands_tools import calculator
from strands.models import BedrockModel
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from bedrock_agentcore.tools.browser_client import browser_session
from nova_act import NovaAct
import json

app = BedrockAgentCoreApp()

# Original custom tool 
@tool
def weather():
    """ Get weather """ 
    return "sunny"

# NEW: Nova Act browser tool
@tool
def browse_web(instruction: str, starting_url: str = "https://google.com"):
    """Use Nova Act to browse the web and complete web-based tasks
    
    Args:
        instruction: What you want to accomplish on the web
        starting_url: URL to start from
    
    Returns:
        Result of the web browsing task
    """
    try:
        with browser_session("us-eat-1") as browser_client:
            ws_url, headers = browser_client.generate_ws_headers()
            
            with NovaAct(
                cdp_endpoint_url=ws_url,
                cdp_headers=headers,
                nova_act_api_key="your-nova-act-api-key",  # Replace with your key
                starting_page=starting_url,
            ) as nova_act:
                result = nova_act.act(instruction)
                return f"Web task completed: {str(result)}"
    except Exception as e:
        return f"Error browsing web: {str(e)}"

# Initialize enhanced agent with Nova Act
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
model = BedrockModel(model_id=model_id)

agent = Agent(
    model=model,
    tools=[calculator, weather, browse_web],  # Now includes web browsing!
    system_prompt="You're a helpful assistant. You can do math calculations, tell the weather, and browse the web to find information."
)

@app.entrypoint
def enhanced_strands_agent(payload):
    """Enhanced agent that can browse the web"""
    user_input = payload.get("prompt")
    print("User input:", user_input)
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

### 🔑 **Key Points for Nova Act in AgentCore Runtime:**

1. **Requirements**: Add `nova-act` to your `requirements.txt`
2. **API Key**: You'll need a Nova Act API key (like you see in the browser tool notebook)
3. **Browser Session**: Uses AgentCore browser tools for web interaction
4. **Deployment**: Works in all deployment modes:
   - ✅ Interactive notebook (local)
   - ✅ Local HTTP server (`agentcore launch --local`)
   - ✅ AWS AgentCore Runtime (deployed)

### 💸 **Additional Costs:**
- **Nova Act API calls** - separate from AWS costs
- **AgentCore Browser Tool** - AWS charges for browser sessions

### 🎯 **Use Cases:**
- Web scraping and data extraction
- Form filling and web automation  
- Research and information gathering
- E-commerce interactions

In [ ]:
# Test 1: Weather tool
response1 = agent("What's the weather like?")
print("🌤️ Weather Query:")
print(response1.message['content'][0]['text'])
print("\n" + "="*50 + "\n")

In [30]:
# Test 2: Calculator tool
response2 = agent("What's 15 * 23 + 7?")
print("🧮 Math Query:")
print(response2.message['content'][0]['text'])
print("\n" + "="*50 + "\n")

I'll calculate that for you using the calculator tool.
Tool #2: calculator


╭────────────────────────────────────────────── Calculation Result ───────────────────────────────────────────────╮
│                                                                                                                 │
│  ╭───────────┬─────────────────────╮                                                                            │
│  │ Operation │ Evaluate Expression │                                                                            │
│  │ Input     │ 15 * 23 + 7         │                                                                            │
│  │ Result    │ 352                 │                                                                            │
│  ╰───────────┴─────────────────────╯                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

The answer is 352.

To break it down: 15 multiplied by 23 equals 345, and then adding 7 gives you 352.🧮 Math Query:
The answer is 352.

To break it down: 15 multiplied by 23 equals 345, and then adding 7 gives you 352.




In [ ]:
# 🚀 Try your own queries! Change the prompt below:
your_query = "Can you calculate the square root of 144 and also tell me the weather?"

response = agent(your_query)
print(f"❓ Your Query: {your_query}")
print(f"🤖 Agent Response:")
print(response.message['content'][0]['text'])

## Different Ways to Run Your Agent Locally

You now have **3 different approaches** for local development:

### 1. **Interactive Notebook** (Above cells) ⚡
- **Fastest for experimentation**
- Test directly in Jupyter
- Perfect for iterating on prompts and tools
- No HTTP overhead

### 2. **Command Line Script** (Cell 5 approach)
```bash
python strands_claude.py '{"prompt": "What is the weather now?"}'
```

### 3. **Local HTTP Server** (What you did with `agentcore launch --local`)
```bash
# Start the server (from your terminal history)
agentcore launch --local

# Test with curl (from another terminal)
curl -X POST http://localhost:8080/invocations \
  -H "Content-Type: application/json" \
  -d '{"prompt": "What is the weather now?"}'
```

**💡 Recommendation:** Start with approach #1 (notebook) for quick testing, then use #3 (local server) to test the HTTP interface before AWS deployment!

## What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

## Deploying the agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial can will the Amazon Bedrock AgentCore Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

### Configure AgentCore Runtime deployment

First we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="60%"/>
</div>

In [12]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "strands_claude_getting_started"
response = agentcore_runtime.configure(
    entrypoint="strands_claude.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

Entrypoint parsed: file=/Users/hsin-weilin/Desktop/projects/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/01-hosting-agent/01-strands-with-bedrock-model/strands_claude.py, bedrock_agentcore_name=strands_claude
Configuring BedrockAgentCore agent: strands_claude_getting_started
Generated Dockerfile: /Users/hsin-weilin/Desktop/projects/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/01-hosting-agent/01-strands-with-bedrock-model/Dockerfile
Generated .dockerignore: /Users/hsin-weilin/Desktop/projects/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/01-hosting-agent/01-strands-with-bedrock-model/.dockerignore
Keeping 'strands_claude_getting_started' as default agent
Bedrock AgentCore configured: /Users/hsin-weilin/Desktop/projects/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/01-hosting-agent/01-strands-with-bedrock-model/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/Users/hsin-weilin/Desktop/projects/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/01-hosting-agent/01-strands-with-bedrock-model/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/Users/hsin-weilin/Desktop/projects/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/01-hosting-agent/01-strands-with-bedrock-model/Dockerfile'), dockerignore_path=PosixPath('/Users/hsin-weilin/Desktop/projects/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/01-hosting-agent/01-strands-with-bedrock-model/.dockerignore'), runtime='Docker', region='us-east-1', account_id='254599367545', execution_role=None, ecr_repository=None, auto_create_ecr=True)

### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [13]:
launch_result = agentcore_runtime.launch()

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'strands_claude_getting_started' to account 254599367545 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: strands_claude_getting_started


Repository doesn't exist, creating new ECR repository: bedrock-agentcore-strands_claude_getting_started


✅ ECR repository available: 254599367545.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-strands_claude_getting_started
Getting or creating execution role for agent: strands_claude_getting_started
Using AWS region: us-east-1, account ID: 254599367545
Role name: AmazonBedrockAgentCoreSDKRuntime-us-east-1-8556fc4504
Role doesn't exist, creating new execution role: AmazonBedrockAgentCoreSDKRuntime-us-east-1-8556fc4504
Starting execution role creation process for agent: strands_claude_getting_started
✓ Role creating: AmazonBedrockAgentCoreSDKRuntime-us-east-1-8556fc4504
Creating IAM role: AmazonBedrockAgentCoreSDKRuntime-us-east-1-8556fc4504
✓ Role created: arn:aws:iam::254599367545:role/AmazonBedrockAgentCoreSDKRuntime-us-east-1-8556fc4504
✓ Execution policy attached: BedrockAgentCoreRuntimeExecutionPolicy-strands_claude_getting_started
Role creation complete and ready for use with Bedrock AgentCore
✅ Execution role available: arn:aws:iam::254599367545:role/AmazonBedrockAgentCoreSDKRunt

### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [14]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: strands_claude_getting_started


'READY'

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [15]:
invoke_response = agentcore_runtime.invoke({"prompt": "How is the weather now?"})
invoke_response

{'ResponseMetadata': {'RequestId': 'e76d3342-014d-4279-aab7-64821a869bca',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 19 Sep 2025 22:10:19 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e76d3342-014d-4279-aab7-64821a869bca',
   'baggage': 'Self=1-68cdd4c6-2e74eba22cf932807b2ed90e,session.id=b6b2210f-ddeb-4dab-8dcd-095d953d1c3e',
   'x-amzn-bedrock-agentcore-runtime-session-id': 'b6b2210f-ddeb-4dab-8dcd-095d953d1c3e',
   'x-amzn-trace-id': 'Root=1-68cdd4c6-50877977440f483001445efc;Self=1-68cdd4c6-2e74eba22cf932807b2ed90e'},
  'RetryAttempts': 0},
 'runtimeSessionId': 'b6b2210f-ddeb-4dab-8dcd-095d953d1c3e',
 'traceId': 'Root=1-68cdd4c6-50877977440f483001445efc;Self=1-68cdd4c6-2e74eba22cf932807b2ed90e',
 'baggage': 'Self=1-68cdd4c6-2e74eba22cf932807b2ed90e,session.id=b6b2210f-ddeb-4dab-8dcd-095d953d1c3e',
 'contentType': 'application/json',
 'statusCode': 200,
 'response': ["It's currentl

### Processing invocation results

We can now process our invocation results to include it in an application

In [22]:
from IPython.display import Markdown, display
import json
#response_text = json.loads(invoke_response['response'][0])
display(Markdown(invoke_response['response'][0]))

It's currently sunny outside! A beautiful day to enjoy some time outdoors if you're able to.

### Invoking AgentCore Runtime with boto3

Now that your AgentCore Runtime was created you can invoke it with any AWS SDK. For instance, you can use the boto3 `invoke_agent_runtime` method for it.

In [23]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is 2+2?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

The answer to 2+2 is 4.

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [24]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

('254599367545.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-strands_claude_getting_started',
 'strands_claude_getting_started-I4bXK0B5GU',
 'bedrock-agentcore-strands_claude_getting_started')

In [25]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
    
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# Congratulations!